In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [10]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
batch_size = 256
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

# 初始化偏值
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

# 卷积层
def conv2d(x, W):
    # x: input tensor of shape [batch, in_height, in_width, in_channels]
    # 采样窗口W: filter/kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    # strides[0] = strides[3] = 1, strides[1]代表x方向的步长, strides[2]代表y方向的步长
    # padding: A string form: 'SAME', 'VALID'
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 池化层
def max_pool_2x2(x):
    # 窗口大小ksize=[1, x, y, 1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 改变x的格式，转为4D向量[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# 初始化第一个卷积层的权值和偏值
W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5的采样窗口，单通道图片，使用32个卷积核，最后可以得到32个卷积平面，一个平面对应一个核
b_conv1 = bias_variable([32]) # 每一个卷积核一个偏值

# 把x_image和权值向量进行卷积，再加上偏值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max pooling

# 初始化第二个卷积层的权值和偏值
W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5的采样窗口，单通道图片，使用32个卷积核，最后可以得到32个卷积平面，一个平面对应一个核
b_conv2 = bias_variable([64]) # 每一个卷积核一个偏值

# 把h_pool1和权值向量进行卷积，再加上偏值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 进行max pooling

# 一张28*28的图片第一次卷积后变为32张28*28平面，第一次池化后变为32张14*14平面
# 第二次卷积后变为64张14*14平面，第二次池化后变为64张7*7平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7 * 7 * 64, 100]) # 上一层有7*7*64个神经元
b_fc1 = bias_variable([1, 100]) # 第一层有1024个节点

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
# 求第一层全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 保留百分之keep_prob的神经元
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层的权值
W_fc2 = weight_variable([100, 10])
b_fc2 = bias_variable([1, 10])
prediction = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# cross entropy
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# 使用梯度下降法
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 判断预测值与真实值是否相同
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            loss_, _ = sess.run([loss, train_step], feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.7})
        test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        print("iter:", epoch, "loss: {:.4f} test accuracy: {:.4f}".format(loss_, test_acc))

iter: 0 loss: 0.5734 test accuracy: 0.9019
iter: 1 loss: 0.3789 test accuracy: 0.9363
iter: 2 loss: 0.2780 test accuracy: 0.9528
iter: 3 loss: 0.2119 test accuracy: 0.9626
iter: 4 loss: 0.1500 test accuracy: 0.9667
iter: 5 loss: 0.1617 test accuracy: 0.9717
iter: 6 loss: 0.1688 test accuracy: 0.9752
iter: 7 loss: 0.1684 test accuracy: 0.9759
iter: 8 loss: 0.0821 test accuracy: 0.9768
iter: 9 loss: 0.0748 test accuracy: 0.9800
iter: 10 loss: 0.1111 test accuracy: 0.9814
iter: 11 loss: 0.0798 test accuracy: 0.9821
iter: 12 loss: 0.0878 test accuracy: 0.9840
iter: 13 loss: 0.0802 test accuracy: 0.9842
iter: 14 loss: 0.1117 test accuracy: 0.9842
iter: 15 loss: 0.0829 test accuracy: 0.9861
iter: 16 loss: 0.0929 test accuracy: 0.9859
iter: 17 loss: 0.0457 test accuracy: 0.9856
iter: 18 loss: 0.0323 test accuracy: 0.9857
iter: 19 loss: 0.0282 test accuracy: 0.9861
iter: 20 loss: 0.0490 test accuracy: 0.9871
